In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

### dataset athlètes

In [ ]:
olympic_athlete = pd.read_json('./data/olympic_athletes.json')

In [ ]:
olympic_athlete.head()

In [ ]:
olympic_athlete.shape

In [ ]:
olympic_athlete.isnull().sum()

### dataset hôtes

In [ ]:
olympic_hosts = pd.read_xml('./data/olympic_hosts.xml')

In [ ]:
olympic_hosts.head()

In [ ]:
olympic_hosts.shape

In [ ]:
olympic_hosts.columns

In [ ]:
olympic_hosts.drop(['index'], axis=1, inplace=True)
olympic_hosts.columns

In [ ]:
olympic_hosts.isnull().sum()

### dataset médailles

In [ ]:
olympic_medals = pd.read_excel('./data/olympic_medals.xlsx')

In [ ]:
olympic_medals.shape

In [ ]:
olympic_medals.isnull().sum()

In [ ]:
olympic_medals.duplicated().sum()

In [ ]:
olympic_medals.columns

In [ ]:
olympic_medals = olympic_medals.rename({'slug_game':'game_slug'},axis=1)
olympic_medals.columns

In [ ]:
olympic_medals.loc[(olympic_medals.duplicated())]

In [ ]:
olympic_medals.loc[(olympic_medals.game_slug == 'paris-1900') & (olympic_medals.discipline_title == 'Polo')]

In [ ]:
olympic_medals.loc[(olympic_medals.game_slug == 'london-1908') & (olympic_medals.discipline_title == 'Polo')]

In [ ]:
olympic_medals.loc[(olympic_medals.game_slug == 'london-1908') & (olympic_medals.discipline_title == 'Hockey') & (olympic_medals.event_title == 'hockey men')] 

In [ ]:
olympic_medals.participant_title.unique()

In [ ]:
olympic_medals.drop(['Unnamed: 0','participant_title', 'athlete_url'], axis=1, inplace=True)

In [ ]:
olympic_medals.tail()

### dataset résultats

In [ ]:
olympic_results = pd.read_html('./data/olympic_results.html')[0]

In [ ]:
olympic_results.head()

In [ ]:
olympic_results.shape

In [ ]:
olympic_results.isnull().sum()

In [ ]:
olympic_results.duplicated().sum()

In [ ]:
olympic_results.loc[(olympic_results.duplicated())]

In [ ]:
olympic_results.value_unit.unique()

In [ ]:
olympic_results.value_type.unique()

In [ ]:
olympic_results.drop(['Unnamed: 0', 'athlete_url'], axis=1, inplace=True)
olympic_results = olympic_results.rename({'slug_game':'game_slug'},axis=1)
olympic_results.columns

### Traitements

- utilisation du dataset `olympic_results` plus complet que `olympic_medals`
- separation des données en deux dataset pour les jo d'hiver et d'été

In [ ]:
jo_types = olympic_hosts[['game_slug','game_season', 'game_year']]

In [ ]:
hosts_results_merge = olympic_results.merge(jo_types, on='game_slug')
hosts_results_merge.head()

In [ ]:
hosts_results_merge.shape

In [ ]:
hosts_results_merge.game_season.unique()

In [ ]:
summer_games_results = hosts_results_merge.loc[(hosts_results_merge.game_season == 'Summer')].copy()
winter_games_results = hosts_results_merge.loc[(hosts_results_merge.game_season == 'Winter')].copy()

In [ ]:
summer_games_results.drop(['game_season'], axis=1, inplace=True)
summer_games_results.head()

In [ ]:
summer_games_results.shape

In [ ]:
summer_games_results.isnull().sum()

In [ ]:
summer_games_results.duplicated().sum()

In [ ]:
dopplers = summer_games_results.loc[(summer_games_results.duplicated())]
dopplers

- calcul du cumul de chaque type de médailles par jo par pays

In [ ]:
summer_games_results.medal_type.unique()

In [ ]:
summer_games_results['medal_type'] = summer_games_results['medal_type'].fillna('None')

In [ ]:
summer_games_results['total_medals'] = summer_games_results['medal_type'].apply(lambda x: 0 if x == 'None' else 1)
summer_games_results['gold_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'GOLD' else 0)
summer_games_results['silver_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'SILVER' else 0)
summer_games_results['bronze_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'BRONZE' else 0)

In [ ]:
summer_games_results

In [ ]:
summer_games_results.country_name.unique()

In [ ]:
historic_medalsbycountry = summer_games_results.groupby(['game_year','country_name']).agg({'total_medals':'sum', 'gold_medals':'sum', 'silver_medals':'sum','bronze_medals':'sum'})

In [ ]:
historic_medalsbycountry = historic_medalsbycountry.sort_values(by=['game_year','total_medals'],ascending=[True, False]).reset_index()

In [ ]:
historic_medalsbycountry

- calcul du nombre de disciplines et d'epreuves disputées par jo par pays

In [ ]:
historic_sportsbycountry = summer_games_results.groupby(['game_year','country_name','discipline_title']).count()
historic_sportsbycountry

In [ ]:
historic_sportsbycountry = historic_sportsbycountry[['event_title']].reset_index()
historic_sportsbycountry = historic_sportsbycountry.rename({'discipline_title':'sports', 'event_title':'epreuves'},axis=1)
historic_sportsbycountry

In [ ]:
historic_sportsbycountry = historic_sportsbycountry.groupby(['game_year','country_name']).agg({'sports':'count'})

In [ ]:
historic_sportsbycountry = historic_sportsbycountry.reset_index()
historic_sportsbycountry

In [ ]:
historic_epreuvesbycountry = summer_games_results.groupby(['game_year','country_name','event_title']).count()
historic_epreuvesbycountry

In [ ]:

historic_epreuvesbycountry = historic_epreuvesbycountry[['discipline_title']].reset_index()
historic_epreuvesbycountry = historic_epreuvesbycountry.rename({'event_title':'epreuves', 'discipline_title':'participation'},axis=1)
historic_epreuvesbycountry = historic_epreuvesbycountry.groupby(['game_year','country_name']).agg({'epreuves':'count'})
historic_epreuvesbycountry = historic_epreuvesbycountry.reset_index()
historic_epreuvesbycountry

In [ ]:
historic_olympic_data = historic_medalsbycountry.merge(historic_sportsbycountry, on=['game_year','country_name'])
historic_olympic_data = historic_olympic_data.merge(historic_epreuvesbycountry, on=['game_year','country_name'])
historic_olympic_data = historic_olympic_data.sort_values(by=['game_year','total_medals'],ascending=[True, False])
historic_olympic_data

### Petite analyse story telling : classement des pays par JO en termes de total de médaille gagnés

In [ ]:
historic_olympic_data.game_year.dtype

In [ ]:
historic_olympic_data.game_year.unique()

In [ ]:
historic_olympic_data.country_name.unique()

In [ ]:
historic_olympic_data.loc[(historic_olympic_data.country_name=='German Democratic Republic (Germany)')]

In [ ]:
historic_olympic_data.loc[(historic_olympic_data.country_name=='ROC')]
#historic_olympic_data.loc[(historic_olympic_data.game_year==1896)]

In [ ]:
historic_olympic_data.loc[(historic_olympic_data.country_name=='Unified Team')]

In [ ]:
historic_olympic_data.loc[(historic_olympic_data.game_year==1992)]['country_name'].unique()

In [ ]:
fig = px.bar(historic_olympic_data, 
             x="country_name", 
             y="total_medals", 
             animation_frame="game_year", 
             hover_name="country_name")

fig.update_layout(
        title="Country ranked on Total medals since 1896",
        xaxis_title="Country",
        yaxis_title="Total medals",
        showlegend=False)

### La même sur les 30 premiers pays par JO

In [ ]:
df_top30 = historic_olympic_data.loc[(historic_olympic_data.game_year==1896)].head(30)
for year in [1900, 1904, 1908, 1912, 1920, 1924, 1928, 1932, 1936, 1948,
       1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992,
       1996, 2000, 2004, 2008, 2012, 2016, 2020]:
    df_temp = historic_olympic_data.loc[(historic_olympic_data.game_year==year)].head(30)
    df_top30 = pd.concat([df_top30, df_temp], ignore_index=True)


In [ ]:
fig = px.bar(df_top30, 
             x="country_name", 
             y="total_medals", 
             animation_frame="game_year", 
             hover_name="country_name")

fig.update_layout(
        title="Top 30 country on Total medals since 1896",
        xaxis_title="Country",
        yaxis_title="Total medals",
        showlegend=False)

### Traitements : Constitution d'un set d'entrainement pour faire de la regression

- je repars de `summer_games_results` pour créer des colonnes contenant le nombre de participation au JO (à l'instant t), et le nombres de médailles gagnées aux JO précedents (t-1) pour chaque pays et chaque JO
- je merge les résultats avec `historic_olympic_data`

In [ ]:
game_part = summer_games_results.groupby(['country_name','game_year']).agg({'total_medals':'sum', 'gold_medals':'sum', 'silver_medals':'sum','bronze_medals':'sum'})
game_part

In [ ]:
game_p = game_part.reset_index()
game_p_france = game_p.loc[(game_p.country_name=='France')]
game_p_france

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
game_p_france = game_p.loc[(game_p.country_name=='France')]
game_p_france['game_part'] = range(0, game_p_france.shape[0])
game_p_france['prec_game_medal'] = game_p_france['total_medals'].shift(1, fill_value=0)
game_p_france['prec_game_gold'] = game_p_france['gold_medals'].shift(1, fill_value=0)
game_p_france['prec_game_silver'] = game_p_france['silver_medals'].shift(1, fill_value=0)
game_p_france['prec_game_bronze'] = game_p_france['bronze_medals'].shift(1, fill_value=0)
#game_p_france['total_game_medal'] = 0
#game_p_france['total_game_medal'] = game_p_france['total_game_medal'].shift(1, fill_value=0) + game_p_france['total_medals'].shift(1, fill_value=0)
#game_p_france['prec_game_medal'].shift(1, fill_value=0) + game_p_france['prec_game_medal'].shift(2, fill_value=0)
game_p_france

In [ ]:
game_p

In [ ]:
liste_country = list(game_p.country_name.unique())
liste_country.remove('France')
liste_country

In [ ]:
for country in liste_country:
    game_p_temp = game_p.loc[(game_p.country_name==country)]
    game_p_temp['game_part'] = range(0, game_p_temp.shape[0])
    game_p_temp['prec_game_medal'] = game_p_temp['total_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_gold'] = game_p_temp['gold_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_silver'] = game_p_temp['silver_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_bronze'] = game_p_temp['bronze_medals'].shift(1, fill_value=0)
    game_p_france = pd.concat([game_p_france,game_p_temp])

In [ ]:
game_p_france.columns

In [ ]:
game_p_france = game_p_france[['game_year', 'country_name', 'total_medals', 'gold_medals', 'silver_medals', 'bronze_medals', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]

In [ ]:
game_p_france = game_p_france.sort_values(by=['game_year','total_medals'],ascending=[True, False])
game_p_france

In [ ]:
game_p_france = game_p_france[['game_year', 'country_name', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]
game_p_france

In [ ]:
historic_olympic_data = pd.merge(historic_olympic_data, game_p_france, on=['game_year','country_name'])


In [ ]:
historic_olympic_data = historic_olympic_data.sort_values(by=['game_year','total_medals'],ascending=[True, False])

### Separation des données en set d'entrainement (tout avant 2020) et de test (JO 2020)

In [ ]:
# Tambouille pour encoder la colonne country_name
liste_pays = list(historic_olympic_data.country_name.unique())
dict_pays = {}
for pays in enumerate(liste_pays):
    dict_pays[pays[1]] = pays[0]

In [ ]:
dict_pays

In [ ]:
data_all_train = historic_olympic_data.loc[(historic_olympic_data.game_year < 2020)]
data_all_test = historic_olympic_data.loc[(historic_olympic_data.game_year == 2020)]

In [ ]:
X_all_train = data_all_train[['country_name', 'sports', 'epreuves', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]
X_all_test = data_all_test[['country_name', 'sports', 'epreuves', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]

y_all_total_train = data_all_train['total_medals']
y_all_gold_train = data_all_train['gold_medals']
y_all_silver_train = data_all_train['silver_medals']
y_all_bronze_train = data_all_train['bronze_medals']

y_all_total_test = data_all_test['total_medals']
y_all_gold_test = data_all_test['gold_medals']
y_all_silver_test = data_all_test['silver_medals']
y_all_bronze_test = data_all_test['bronze_medals']

In [ ]:
# Encodage des pays
X_all_train = X_all_train.replace(dict_pays)
X_all_test = X_all_test.replace(dict_pays)

### Entrainement et evaluation modèles de regression pour le total de médaille par pays

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = LinearRegression()
model.fit(X_all_train, y_all_total_train)

In [ ]:
predictions = model.predict(X_all_test)

print('Predicted labels : ', np.round(predictions)[:10])
print('Actual labels : ' , y_all_total_test[:10])

In [ ]:
mse = mean_squared_error(y_all_total_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_all_total_test, predictions)
print("R2:", r2)

In [ ]:
result_df_2020_total_medals = pd.DataFrame({'pays': data_all_test['country_name'],'reglog_total_medals_pred':np.round(predictions),'total_medals_truth':data_all_test['total_medals']})

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
result_df_2020_total_medals

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model_1 = DecisionTreeRegressor()
model_1.fit(X_all_train, y_all_total_train)

In [ ]:
predictions_1 = model_1.predict(X_all_test)

print('Predicted labels : ', np.round(predictions_1)[:10])
print('Actual labels : ' , y_all_total_test[:10])

In [ ]:
mse = mean_squared_error(y_all_total_test, predictions_1)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_all_total_test, predictions_1)
print("R2:", r2)

In [ ]:
result_df_2020_total_medals['tree_total_medals_pred'] = np.round(predictions_1)
result_df_2020_total_medals

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model_2 = GradientBoostingRegressor()
model_2.fit(X_all_train, y_all_total_train)

In [ ]:
predictions_2 = model_2.predict(X_all_test)

print('Predicted labels : ', np.round(predictions_2)[:10])
print('Actual labels : ' , y_all_total_test[:10])

In [ ]:
mse = mean_squared_error(y_all_total_test, predictions_2)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_all_total_test, predictions_2)
print("R2:", r2)

In [ ]:
result_df_2020_total_medals['XGB_total_medals_pred'] = np.round(predictions_2)
result_df_2020_total_medals.columns

### Résultats Prédiction tableau du total des médailles par pays

In [ ]:
result_df_2020_total_medals = result_df_2020_total_medals[['pays', 'reglog_total_medals_pred', 'tree_total_medals_pred', 'XGB_total_medals_pred', 'total_medals_truth']]
result_df_2020_total_medals